<a href="https://colab.research.google.com/github/dianaalme46-hue/compling-course/blob/main/w2v_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В этом практикуме мы рассмотрим работу с библиотекой **Gensim** для работы с векторными представлениями текста

Мы рассмотрим
- **Word2Vec** - векторные представления слов
- **FastText** - улучшенные представления с учетом морфологии  
- **Doc2Vec** - векторные представления документов


In [ ]:
!pip install gensim

import gensim
import gensim.downloader as api
from gensim.models import Word2Vec, FastText, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import numpy as np

## Часть 1: Word2Vec

### Что такое Word2Vec?

Word2Vec преобразует слова в векторы чисел так, что семантически похожие слова оказываются близко в векторном пространстве.

**Два основных алгоритма:**
- **CBOW** - предсказывает слово по контексту
- **Skip-gram** - предсказывает контекст по слову

**Загрузка предобученной модели**

In [ ]:
w2v_model = api.load('glove-wiki-gigaword-100')

print(f"Размер словаря: {len(w2v_model.key_to_index)}")
print(f"Размерность векторов: {w2v_model.vector_size}")

[==================================================] 100.0% 128.1/128.1MB downloaded
Размер словаря: 400000
Размерность векторов: 100


Найдите документацию `gensim`: какие датасеты кроме `glove-wiki-gigaword-100` доступны в библиотеке?

В библиотеке также доступны fasttext-wiki-news-subwords-300, glove-twitter-100, conceptnet-numberbatch-17-06-300, glove-wiki-gigaword-300, word2vec-ruscorpora-300 и другие датасеты

Выберите 3 датасета и кратко опишите их (источник данных, примерный объем, зачем такой датасет может использоваться)

1. glove-wiki-gigaword-300: 376 МБ, 400 т. токенов. Источник - Wikipedia 2014 и Gigaword. Может использоваться для генерации информационных сообщений (боты), для семантической обработки текста
2. word2vec-google-news-300: 1662 МБ, 100 млрд. слов. Источник - Google News Dataset. Может использоваться для анализа новостных текстов (ключевые слова, стиль, речевые штампы, конструкции, синтаксис)
3. glove-twitter-50: 199 МБ, 1.2 млн. слов. Источник - Twitter. Может использоваться для улучшения системы безопасности, которые выявляют тексты с нежелательными элементами (цензура), для анализа современного Интернет-языка.

**Базовые операции с векторами**

In [ ]:
# Получаем вектор слова
vector = w2v_model['computer']
print(f"Вектор слова 'computer': {vector[:5]}...")  # Показываем первые 5 чисел

# Вычисляем схожесть между словами
similarity = w2v_model.similarity('computer', 'laptop')
print(f"Схожесть 'computer' и 'laptop': {similarity:.4f}")

Вектор слова 'computer': [-0.16298   0.30141   0.57978   0.066548  0.45835 ]...
Схожесть 'computer' и 'laptop': 0.7024


**Поиск похожих слов**

In [ ]:
# Находим похожие слова
similar_words = w2v_model.most_similar('python', topn=5)
print("Слова, похожие на 'python':")
for word, score in similar_words:
    print(f"  {word}: {score:.4f}")

*Ваш ответ здесь*

**Задание**

1. Загрузите любой датасет из gensim на ваш выбор

In [27]:
w2v_model = api.load('glove-twitter-50') #к сожалению, на моем ноутбуке не загружаются датасеты, поэтому не могу проверить работу кода и работу следующей ячейки (с функцией, которая возвращает близкие слова)

print(f"Размер словаря: {len(w2v_model.key_to_index)}")
print(f"Размерность векторов: {w2v_model.vector_size}")

Размер словаря: 1193514
Размерность векторов: 50


2. Напишите функцию, которая принимает на вход любое слово и вовращает 10 наиболее близких по вектору слов

In [28]:
u_w = input() #вводится любое слово
similar_words = w2v_model.most_similar(u_w, topn=10) #находим 10 похожих слов
print(f"Слова, похожие на {u_w}:")
for word, score in similar_words:
    print(f"  {word}: {score:.4f}")

KeyboardInterrupt: Interrupted by user

3. Обучите модель Word2Vec на тестовом датасете из ячейки ниже

Примените следующие настройки:

- размер вектора: 50
- размер окна: 3
- минимальная частота слова: 1
- потоков: 2
- использовать skip-gram

In [29]:
cooking_sentences = [
    ['варить', 'суп', 'овощи', 'морковь', 'картофель'],
    ['жарить', 'курица', 'сковорода', 'масло', 'специи'],
    ['печь', 'хлеб', 'мука', 'дрожжи', 'духовка'],
    ['резать', 'овощи', 'салат', 'помидоры', 'огурцы'],
    ['смешивать', 'ингредиенты', 'тесто', 'яйца', 'молоко'],
    ['варить', 'паста', 'вода', 'соль', 'соус'],
    ['гриль', 'мясо', 'овощи', 'уголь', 'барбекю'],
    ['тушить', 'говядина', 'горшок', 'вино', 'травы'],
    ['запекать', 'рыба', 'лимон', 'духовка', 'фольга'],
    ['готовить', 'завтрак', 'яичница', 'бекон', 'тост'],
    ['месить', 'тесто', 'пирог', 'начинка', 'яблоки'],
    ['кипятить', 'вода', 'чай', 'кофе', 'чашка'],
    ['мариновать', 'мясо', 'соус', 'специи', 'холодильник'],
    ['взбивать', 'сливки', 'сахар', 'десерт', 'торт'],
    ['парить', 'овощи', 'здоровое', 'питание', 'брокколи']
]

In [49]:
model = cbow_model = Word2Vec(
    cooking_sentences,
    vector_size=50,      # размерность векторов
    window=3,             # размер контекстного окна
    min_count=1,          # минимальная частота слова
    workers=2,            # количество ядер
    sg=1                  # 0 = CBOW, 1 = Skip-Gram
)

In [50]:
print(f"Слова в словаре: {list(model.wv.key_to_index.keys())[:10]}...")

Слова в словаре: ['овощи', 'мясо', 'соус', 'вода', 'тесто', 'духовка', 'специи', 'варить', 'брокколи', 'питание']...


4. Проверьте модель

In [51]:
# Проверяем похожие слова в кулинарной тематике
try:
    similar = model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

Слова, похожие на 'варить':
  вино: 0.2398
  ингредиенты: 0.2172
  хлеб: 0.1938
  брокколи: 0.1846
  кипятить: 0.1711


In [52]:
# Найдите слова, похожие на "духовка"
try:
    similar = model.wv.most_similar('духовка', topn=5)
    print("Слова, похожие на 'духовка':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'духовка' не найдено в словаре")

# Найдите слова, похожие на "овощи"
try:
    similar = model.wv.most_similar('овощи', topn=5)
    print("Слова, похожие на 'овощи':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'овощи' не найдено в словаре")

Слова, похожие на 'духовка':
  ингредиенты: 0.3199
  десерт: 0.3064
  холодильник: 0.2705
  питание: 0.2243
  пирог: 0.2142
Слова, похожие на 'овощи':
  мариновать: 0.2716
  хлеб: 0.2691
  гриль: 0.2546
  фольга: 0.2409
  сахар: 0.2108


## Часть 2: FastText

FastText улучшает Word2Vec, рассматривая слова как наборы символов (n-грамм). Это позволяет работать с редкими словами и опечатками

5. Обучите FastText на корпусе текстов из пункта 3. Используйте код ниже

In [37]:
ft_model = FastText(
    sentences=cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2
)



In [38]:
print(f"Слова в словаре: {list(model.wv.key_to_index.keys())[:10]}...")

Слова в словаре: ['овощи', 'мясо', 'соус', 'вода', 'тесто', 'духовка', 'специи', 'варить', 'брокколи', 'питание']...


6. Найдите слова, похожие на "варить", "духовка" и "овощи" с помощью обученной модели. Используйте код из пункта 4

In [39]:
try:
    similar = ft_model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

try:
    similar = ft_model.wv.most_similar('духовка', topn=5)
    print("Слова, похожие на 'духовка':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'духовка' не найдено в словаре")

try:
    similar = ft_model.wv.most_similar('духовка', topn=5)
    print("Слова, похожие на 'духовка':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'духовка' не найдено в словаре")

Слова, похожие на 'варить':
  жарить: 0.5353
  парить: 0.4805
  месить: 0.3541
  тушить: 0.3405
  специи: 0.2622
Слова, похожие на 'духовка':
  взбивать: 0.4565
  лимон: 0.3561
  салат: 0.3050
  курица: 0.3041
  тост: 0.2944
Слова, похожие на 'духовка':
  взбивать: 0.4565
  лимон: 0.3561
  салат: 0.3050
  курица: 0.3041
  тост: 0.2944


7. Сравните модели

Дана функция для сравнения Word2Vec и FastText

Придумайте 3 слова с опечатками и проверьте, найдет ли их FastText и Word2Vec

In [40]:
def compare_models(word):
    """Сравнивает представления слова в разных моделях"""
    print(f"\nСравнение для слова: '{word}'")

    # Word2Vec
    try:
        w2v_similar = model.wv.most_similar(word, topn=2)
        print(f"  Word2Vec: {[w for w, _ in w2v_similar]}")
    except KeyError:
        print(f"  Word2Vec: слово не найдено")

    # FastText
    try:
        ft_similar = ft_model.wv.most_similar(word, topn=2)
        print(f"  FastText: {[w for w, _ in ft_similar]}")
    except KeyError:
        print(f"  FastText: слово не найдено")

# Сравниваем для разных слов: пишу слова "вода" неправильно, слова "тесто" и "соус" с опечатками
compare_models('вада')
compare_models('темто')
compare_models('совс')
#с задачей справился только FastText, нашел два слова из трех - "вода" и "тесто"


Сравнение для слова: 'вада'
  Word2Vec: слово не найдено
  FastText: ['уголь', 'вода']

Сравнение для слова: 'темто'
  Word2Vec: слово не найдено
  FastText: ['тесто', 'гриль']

Сравнение для слова: 'совс'
  Word2Vec: слово не найдено
  FastText: ['овощи', 'ингредиенты']


## Часть 3: Doc2Vec

Doc2Vec расширяет Word2Vec для создания векторных представлений целых документов (предложений, абзацев, статей)

In [41]:
# Создаем размеченные документы
documents = [
    "machine learning is interesting",
    "deep learning uses neural networks",
    "python programming for data science",
    "artificial intelligence is amazing",
    "computer vision processes images"
]

# Преобразуем в формат TaggedDocument
tagged_docs = []
for i, doc in enumerate(documents):
    tokens = doc.split()
    tagged_doc = TaggedDocument(words=tokens, tags=[f"doc_{i}"])
    tagged_docs.append(tagged_doc)

print("Размеченные документы:")
for doc in tagged_docs[:3]:
    print(f"  Слова: {doc.words}")
    print(f"  Тег: {doc.tags}")

Размеченные документы:
  Слова: ['machine', 'learning', 'is', 'interesting']
  Тег: ['doc_0']
  Слова: ['deep', 'learning', 'uses', 'neural', 'networks']
  Тег: ['doc_1']
  Слова: ['python', 'programming', 'for', 'data', 'science']
  Тег: ['doc_2']


In [42]:
# Обучаем Doc2Vec
doc_model = Doc2Vec(
    documents=tagged_docs,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    epochs=20
)

print("Doc2Vec модель обучена!")
print(f"Количество документов: {len(doc_model.dv.key_to_index)}")

Doc2Vec модель обучена!
Количество документов: 5


In [43]:
# Получаем вектор документа
doc_vector = doc_model.dv["doc_0"]
print(f"Вектор документа doc_0: {doc_vector[:5]}...")

# Находим похожие документы
similar_docs = doc_model.dv.most_similar("doc_0", topn=2)
print("\nДокументы, похожие на doc_0:")
for doc_tag, similarity in similar_docs:
    doc_id = int(doc_tag.split('_')[1])
    print(f"  {doc_tag}: {similarity:.4f}")
    print(f"    Текст: {documents[doc_id]}")

Вектор документа doc_0: [-0.01057    -0.01198188 -0.01982618  0.01710627  0.00710373]...

Документы, похожие на doc_0:
  doc_1: 0.2735
    Текст: deep learning uses neural networks
  doc_2: 0.1275
    Текст: python programming for data science


In [44]:
# Сравниваем схожесть документов
def compare_documents(doc1_id, doc2_id):
    similarity = doc_model.dv.similarity(f"doc_{doc1_id}", f"doc_{doc2_id}")
    print(f"Схожесть doc_{doc1_id} и doc_{doc2_id}: {similarity:.4f}")
    print(f"  doc_{doc1_id}: {documents[doc1_id]}")
    print(f"  doc_{doc2_id}: {documents[doc2_id]}")

compare_documents(0, 1)  # machine learning vs deep learning
compare_documents(0, 3)  # machine learning vs AI

Схожесть doc_0 и doc_1: 0.2735
  doc_0: machine learning is interesting
  doc_1: deep learning uses neural networks
Схожесть doc_0 и doc_3: -0.0822
  doc_0: machine learning is interesting
  doc_3: artificial intelligence is amazing


8. Сравните схожесть doc_2 и doc_4

In [45]:
compare_documents(2, 4) #применяю функцию к документам

Схожесть doc_2 и doc_4: -0.0362
  doc_2: python programming for data science
  doc_4: computer vision processes images


9. Найдите самый похожий документ на doc_1

In [46]:
compare_documents(1, 0) #применяю функцию к документам
compare_documents(1, 2)
compare_documents(1, 3)
compare_documents(1, 4)
# самый похожий документ - doc_0

Схожесть doc_1 и doc_0: 0.2735
  doc_1: deep learning uses neural networks
  doc_0: machine learning is interesting
Схожесть doc_1 и doc_2: -0.0573
  doc_1: deep learning uses neural networks
  doc_2: python programming for data science
Схожесть doc_1 и doc_3: 0.2031
  doc_1: deep learning uses neural networks
  doc_3: artificial intelligence is amazing
Схожесть doc_1 и doc_4: -0.2546
  doc_1: deep learning uses neural networks
  doc_4: computer vision processes images


10. Выберите любую из трёх моделей. Обучите модели с разной размерностью (10, 50, 100). Продемонстрируйте качество их работы на примере поиска похожих слов (выберите любые 3 примера, соответствующих тематике корпуса из пункта 4)

In [48]:
cooking_sentences = [
    ['варить', 'суп', 'овощи', 'морковь', 'картофель'],
    ['жарить', 'курица', 'сковорода', 'масло', 'специи'],
    ['печь', 'хлеб', 'мука', 'дрожжи', 'духовка'],
    ['резать', 'овощи', 'салат', 'помидоры', 'огурцы'],
    ['смешивать', 'ингредиенты', 'тесто', 'яйца', 'молоко'],
    ['варить', 'паста', 'вода', 'соль', 'соус'],
    ['гриль', 'мясо', 'овощи', 'уголь', 'барбекю'],
    ['тушить', 'говядина', 'горшок', 'вино', 'травы'],
    ['запекать', 'рыба', 'лимон', 'духовка', 'фольга'],
    ['готовить', 'завтрак', 'яичница', 'бекон', 'тост'],
    ['месить', 'тесто', 'пирог', 'начинка', 'яблоки'],
    ['кипятить', 'вода', 'чай', 'кофе', 'чашка'],
    ['мариновать', 'мясо', 'соус', 'специи', 'холодильник'],
    ['взбивать', 'сливки', 'сахар', 'десерт', 'торт'],
    ['парить', 'овощи', 'здоровое', 'питание', 'брокколи']
]

ft_model = FastText(
    sentences=cooking_sentences,
    vector_size=10, #задаю размереность векторов
    window=3,
    min_count=1,
    workers=2
)
print('Размерность: 10') #вывожу размерность, чтобы визуально разграничить результаты между собой
try:
    similar = ft_model.wv.most_similar('курица', topn=5)
    print("Слова, похожие на 'курица':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'курица' не найдено в словаре")

try:
    similar = ft_model.wv.most_similar('хлеб', topn=5)
    print("Слова, похожие на 'хлеб':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'хлеб' не найдено в словаре")

try:
    similar = ft_model.wv.most_similar('завтрак', topn=5)
    print("Слова, похожие на 'завтрак':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'завтрак' не найдено в словаре")

ft_model = FastText(
    sentences=cooking_sentences,
    vector_size=50, #задаю размерность векторов
    window=3,
    min_count=1,
    workers=2
)
print('Размерность: 50') #вывожу размерность, чтобы визуально разграничить результаты между собой
try:
    similar = ft_model.wv.most_similar('курица', topn=5)
    print("Слова, похожие на 'курица':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'курица' не найдено в словаре")

try:
    similar = ft_model.wv.most_similar('хлеб', topn=5)
    print("Слова, похожие на 'хлеб':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'хлеб' не найдено в словаре")

try:
    similar = ft_model.wv.most_similar('завтрак', topn=5)
    print("Слова, похожие на 'завтрак':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'завтрак' не найдено в словаре")

ft_model = FastText(
    sentences=cooking_sentences,
    vector_size=100, #задаю размерность векторов
    window=3,
    min_count=1,
    workers=2
)
print('Размерность: 100') #вывожу размерность, чтобы визуально разграничить результаты между собой
try:
    similar = ft_model.wv.most_similar('курица', topn=5)
    print("Слова, похожие на 'курица':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'курица' не найдено в словаре")

try:
    similar = ft_model.wv.most_similar('хлеб', topn=5)
    print("Слова, похожие на 'хлеб':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'хлеб' не найдено в словаре")

try:
    similar = ft_model.wv.most_similar('завтрак', topn=5)
    print("Слова, похожие на 'завтрак':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'завтрак' не найдено в словаре")
#на мой взгляд, увеличение размерности ухудшает вывод, так как чем больше размерность, тем больше в результате менее тематически и категориально похожих слов: при размерности 10 похожие слова у "хлеба" "завтрак", "бекон", "суп", при размерности 100 - "огурцы", "смешивать".

Размерность: 10
Слова, похожие на 'курица':
  яичница: 0.7073
  морковь: 0.6829
  говядина: 0.6016
  холодильник: 0.4984
  масло: 0.4330
Слова, похожие на 'хлеб':
  завтрак: 0.8581
  бекон: 0.5372
  суп: 0.4872
  яйца: 0.4838
  барбекю: 0.4425
Слова, похожие на 'завтрак':
  хлеб: 0.8581
  бекон: 0.5933
  говядина: 0.5557
  пирог: 0.5231
  картофель: 0.4467
Размерность: 50
Слова, похожие на 'курица':
  брокколи: 0.4155
  духовка: 0.3041
  запекать: 0.2837
  взбивать: 0.2663
  месить: 0.2643
Слова, похожие на 'хлеб':
  молоко: 0.3031
  паста: 0.2314
  чай: 0.2239
  ингредиенты: 0.1773
  специи: 0.1738
Слова, похожие на 'завтрак':
  травы: 0.3880
  резать: 0.3389
  тост: 0.3258
  суп: 0.2591
  варить: 0.2032
Размерность: 100
Слова, похожие на 'курица':
  яичница: 0.3198
  вино: 0.1821
  тушить: 0.1776
  пирог: 0.1453
  морковь: 0.1452
Слова, похожие на 'хлеб':
  бекон: 0.2122
  картофель: 0.2108
  брокколи: 0.1808
  огурцы: 0.1705
  смешивать: 0.1443
Слова, похожие на 'завтрак':
  печь: 0